# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

gmaps.configure(api_key=g_key)

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
cities_df = pd.read_csv("../WeatherPY/output_data/cities.csv")
cities_df.drop(cities_df.columns[[0]], axis=1, inplace=True)
cities_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,fairbanks,64.8378,-147.7164,257.59,77,75,0.11,US,1613422203
1,mataura,-46.1927,168.8643,285.37,77,80,0.45,NZ,1613421981
2,evensk,61.9500,159.2333,261.12,88,100,1.52,RU,1613422359
3,hobart,-42.8794,147.3294,287.15,88,75,3.60,AU,1613421904
4,ushuaia,-54.8000,-68.3000,289.15,48,40,7.72,AR,1613421966
...,...,...,...,...,...,...,...,...,...
554,maningrida,-12.0667,134.2667,298.73,92,100,3.03,AU,1613423015
555,galle,6.0367,80.2170,297.64,85,16,0.67,LK,1613423016
556,requena,39.4883,-1.1004,287.15,82,90,1.88,ES,1613423017
557,acapulco,16.8634,-99.8901,301.15,54,1,2.57,MX,1613422767


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [11]:
locations = cities_df[["Lat", "Lng"]]
humidity = cities_df['Humidity']
fig = gmaps.figure(zoom_level = 2, center = (46.0, -5.0))
heat_layer = gmaps.heatmap_layer(locations, weights = humidity , dissipating = False, max_intensity = 100, point_radius =4)
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [4]:
cleaned_cities_df = cities_df.loc[ (cities_df["Cloudiness"] < 10) & (cities_df["Max Temp"] < 302) & (cities_df["Max Temp"] > 299), :]
cleaned_cities_df.reset_index(drop=True, inplace=True)
cleaned_cities_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,rikitea,-23.1203,-134.9692,299.58,79,3,6.59,PF,1613421965
1,butaritari,3.0707,172.7902,299.46,79,0,6.64,KI,1613421950
2,avarua,-21.2078,-159.7750,301.15,78,0,3.09,CK,1613422022
3,saint-philippe,-21.3585,55.7679,299.15,69,0,2.57,RE,1613422085
4,ugoofaaru,5.6667,73.0000,300.26,65,0,2.12,MV,1613422482
5,saint-pierre,-21.3393,55.4781,299.15,69,0,2.57,RE,1613422563
6,saint-joseph,-21.3667,55.6167,299.15,69,0,2.57,RE,1613422585
7,pisco,-13.7000,-76.2167,299.15,61,0,6.17,PE,1613421864
8,iquique,-20.2208,-70.1431,299.15,44,0,9.26,CL,1613422744
9,key west,24.5557,-81.7826,301.15,74,1,6.17,US,1613422531


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [5]:
hotel_df = cleaned_cities_df
hotel_df["Hotel Name"] = ''


/Users/ainashtoleu/opt/anaconda3/envs/PythonData/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [6]:
base_url = 'https://maps.googleapis.com/maps/api/place/nearbysearch/json'

params = {
    "radius": 5000,
    "keyword": "hotel",
    "key": g_key
    }

for index, row in hotel_df.iterrows():
    
    Lat = row["Lat"]
    Lng = row["Lng"]
    
    params["location"] = f"{Lat},{Lng}",
    
    print(f'Retrieving results for Index:{index}:{Lat},{Lng}.')
    response = requests.get(base_url, params=params).json()
    
    results = response['results']
    
    try:
        print(f"First Hotel is {results[0]['name']}.")
        hotel_df.loc[index, 'Hotel Name'] = results[0]['name']
    except:
        print("Missing Hotel ... skipping.")
        
    print("--------------")    



    

Retrieving results for Index:0:-23.1203,-134.9692.
First Hotel is Pension Bianca & Benoit.
--------------
Retrieving results for Index:1:3.0707,172.7902.


/Users/ainashtoleu/opt/anaconda3/envs/PythonData/lib/python3.6/site-packages/pandas/core/indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


Missing Hotel ... skipping.
--------------
Retrieving results for Index:2:-21.2078,-159.775.
First Hotel is Sunrise Beach Bungalows.
--------------
Retrieving results for Index:3:-21.3585,55.7679.
First Hotel is Les Embruns Du Baril.
--------------
Retrieving results for Index:4:5.6667,73.0.
First Hotel is Ungoofaaru Inn.
--------------
Retrieving results for Index:5:-21.3393,55.4781.
First Hotel is Villa Delisle, Hotel & Spa.
--------------
Retrieving results for Index:6:-21.3667,55.6167.
First Hotel is "Plantation Bed and Breakfast.
--------------
Retrieving results for Index:7:-13.7,-76.2167.
First Hotel is HOSTAL MIRAMAR.
--------------
Retrieving results for Index:8:-20.2208,-70.1431.
First Hotel is Hotel Terrado Cavancha.
--------------
Retrieving results for Index:9:24.5557,-81.7826.
First Hotel is Sunset Key Cottages.
--------------
Retrieving results for Index:10:16.65,-98.9833.
First Hotel is Hotel Picolino.
--------------
Retrieving results for Index:11:-20.2333,164.0167.
Mi

In [7]:
hotel_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
0,rikitea,-23.1203,-134.9692,299.58,79,3,6.59,PF,1613421965,Pension Bianca & Benoit
1,butaritari,3.0707,172.7902,299.46,79,0,6.64,KI,1613421950,
2,avarua,-21.2078,-159.7750,301.15,78,0,3.09,CK,1613422022,Sunrise Beach Bungalows
3,saint-philippe,-21.3585,55.7679,299.15,69,0,2.57,RE,1613422085,Les Embruns Du Baril
4,ugoofaaru,5.6667,73.0000,300.26,65,0,2.12,MV,1613422482,Ungoofaaru Inn
5,saint-pierre,-21.3393,55.4781,299.15,69,0,2.57,RE,1613422563,"Villa Delisle, Hotel & Spa"
6,saint-joseph,-21.3667,55.6167,299.15,69,0,2.57,RE,1613422585,"""Plantation Bed and Breakfast"
7,pisco,-13.7000,-76.2167,299.15,61,0,6.17,PE,1613421864,HOSTAL MIRAMAR
8,iquique,-20.2208,-70.1431,299.15,44,0,9.26,CL,1613422744,Hotel Terrado Cavancha
9,key west,24.5557,-81.7826,301.15,74,1,6.17,US,1613422531,Sunset Key Cottages


In [8]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [19]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(marker_locations, info_box_content = hotel_info)
fig = gmaps.figure()
fig.add_layer(markers)

# Display figure
fig


Figure(layout=FigureLayout(height='420px'))